In [1]:
target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
                 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 
                 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
                 'Swan Ganz Catheter Present']

In [2]:
%%capture
# ====================================================
# Library
# ====================================================
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
sys.path.append('../input/pytorch-images-seresnet')
import os
import copy
import math
import ast
import time
import random
import shutil

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.utils import check_random_state
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, train_test_split

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import torchvision

import albumentations
from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose, HueSaturationValue, CoarseDropout
    )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

from tqdm.notebook import tqdm
import timm

import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Load in Models 

In [3]:
class BatchNormBlock(nn.Module):
    def __init__(self, in_features, out_features, kernel_size, padding, stride, groups):
        super().__init__()
        self.conv = nn.Conv2d(in_features, out_features, kernel_size = kernel_size, padding = padding, stride= stride, groups = groups)
        self.bn = nn.BatchNorm2d(out_features)
    def forward(self, x):
        return self.bn(F.relu(self.conv(x), inplace = True)) 

In [4]:
class BottleNeck(nn.Module):
    def __init__(self, in_dim, inner_dim):
        '''
        in_features, out_features, kernel_size, padding, stride, groups
        '''
        super().__init__()
        self.in_dim = in_dim
        self.inner_dim = inner_dim
        self.squeeze = BatchNormBlock(self.in_dim, self.inner_dim, 1, 0, 1, 1)
        self.process = BatchNormBlock(self.inner_dim, self.inner_dim, 3, 1, 1, 1)
        self.expand = BatchNormBlock(self.inner_dim, self.in_dim, 1, 0, 1, 1)
        self.bn = nn.BatchNorm2d(self.in_dim)
        self.gamma = nn.Parameter(torch.zeros(1, device = device))
        #self.gamma.requires_grad = False
    def forward(self, x):
        return self.bn(self.expand(self.process(self.squeeze(x)))) * self.gamma + x

In [5]:
class ModifiedResNetAlpha(nn.Module):
    def freeze(self, layer):
        for parameter in layer.parameters():
            parameter.requires_grad = False
    def __init__(self, num_classes, device, drop_prob = 0.2):
        # modified ResNet Student, with additional processing and one less attention head for maximal performance
        super().__init__()
        self.device = device
        self.num_classes = num_classes
        self.model = timm.create_model("resnet200d_320", pretrained = True, features_only = True)
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        
        self.conv1 = self.model.conv1
        self.bn1 = self.model.bn1
        
        self.layer1 = self.model.layer1
        
        self.layer2 = self.model.layer2
        
        self.layer3 = self.model.layer3
        
        # Freeze Initial Layers
        #self.freeze(self.layer1)
        #self.freeze(self.conv1)
        #self.freeze(self.bn1)
        #self.freeze(self.layer2)
        #self.freeze(self.layer3) 
        
        self.layer4 = self.model.layer4 # Extract layers from ResNet
        
        self.layer2_5 = nn.Sequential(*[
            BottleNeck(512, 128) for i in range(12) # 18 BottleNeck Blocks, Really Really Cheap Operation so use wisely.
        ])
        
        self.layer2_att = nn.Identity()#CBAMAttention(512, 256, 2)
        self.se2 = nn.Identity()#CBAMSqueezeExcite(512, 256)
        
        self.layer3_att = nn.Identity()#CBAMAttention(1024, 512, 2)
        self.se3 = nn.Identity()#CBAMSqueezeExcite(1024, 256) 
        # Custom Layer 3.5(Between layer 3 and 4)
        
        self.layer3_5 = nn.Sequential(*[BottleNeck(1024, 256) for i in range(15)]) #+
            #[nn.MaxPool2d(kernel_size = 3, padding = 1, stride = 2)] + 
            #[BottleNeck(1024, 256) for i in range(12)])

        
        self.layer4_att = nn.Identity()#CBAMAttention(1024, 512, 2)
        self.se4 = nn.Identity()#CBAMSqueezeExcite(1024, 256) 
        
        self.Dropout = nn.Dropout(drop_prob)
        self.global_avg = nn.AdaptiveAvgPool2d((1, 1))
        self.Linear = nn.Linear(2048, self.num_classes)
       
        
    def forward(self, x):
        B, _, _, _ = x.shape
        base_features = self.model(x)[-3] # (B, 512, 80, 80)
        # Process through Layer 2.5
        layer2_5 = self.layer2_5(base_features) # (B, 512, 80, 80)
        
        attention2 = self.layer2_att(layer2_5)
        excited_features2 = self.se2(attention2)
    
        layer3 = self.layer3(excited_features2) # (B, 1024, 40, 40)
        
        attention3 = self.layer3_att(layer3)
        excited_features3 = self.se3(attention3)
        
        layer3_5 = self.layer3_5(excited_features3)
        attention4 = self.layer4_att(layer3_5)
        excited_features4 = self.se4(attention4) # (B, 1024, 10, 10)
        
        layer4 = self.layer4(excited_features4) # (B, 2048, 5, 5)
        # Pooled Features
        avg_pooled = torch.squeeze(self.global_avg(layer4)) # (B, 2048)
        dropped_features = self.Dropout(avg_pooled) # (B, 2048)
        
        logits = self.Linear(dropped_features) # (B, 11)
        
        return logits, excited_features2, excited_features3, excited_features4, layer4, avg_pooled 

In [19]:
class StudentSolver(nn.Module):
    def __init__(self, student):
        super().__init__()
        self.student = student
    def forward(self, x):
        '''
        Runs Inference on the Student Model, performing a sigmoid on the logits
        '''
        self.eval()
        with torch.no_grad():
            logits, _, _, _, _, _ = self.student(x)
            return logits

Load in the Data

In [7]:
IMAGE_SIZE = 320
test_transforms = Compose([
            Resize(IMAGE_SIZE, IMAGE_SIZE),
            Normalize(),
            ToTensorV2(),
        ])

In [8]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, df, transforms):
        self.df = df 
        self.file_names = df.index.values
        self.transforms = transforms 
        self.testing_path = "../input/ranzcr-clip-catheter-line-classification/test/"
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        test_id= self.file_names[idx]
        test_path = self.testing_path + test_id + '.jpg'
        # load in the image
        image = cv2.imread(test_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Test Transforms
        transformed_image = self.transforms(image = image)['image']
        return transformed_image, test_id
        

In [9]:
test_data = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/sample_submission.csv", index_col = "StudyInstanceUID")
test_dataset = TestDataset(test_data, test_transforms)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 8)

# Load Models into Memory

In [10]:
class ResNet200D(nn.Module):
    def __init__(self, model_name='resnet200d_320'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output

class SeResNet152D(nn.Module):
    def __init__(self, model_name='seresnet152d'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output

In [11]:
# CONSTANTS
RESNET200D_PATH = '../input/resnet200d-public/resnet200d_320_CV9632.pth'
SERESNET_PATH = '../input/seresnet152d-cv9615/seresnet152d_320_CV96.15.pth'
CUSTOM_MODEL_PATH = "../input/stage3/FinalModel.pth"
SERESNET_WEIGHT = 1
RESNET200D_WEIGHT = 2
CUSTOMMODEL_WEIGHT = 1
DIVIDE = 4

In [12]:
%%capture
resnet200D = ResNet200D()
resnet200D.load_state_dict(torch.load(RESNET200D_PATH)['model'])
resnet200D.to(device)
seresnet152 = SeResNet152D()
seresnet152.load_state_dict(torch.load(SERESNET_PATH)['model'])
seresnet152.to(device)

In [20]:
%%capture
# Load Custom Model Here.
CustomModel = StudentSolver(ModifiedResNetAlpha(11, device, drop_prob = 0.0))
CustomModel.to(device)
CustomModel.load_state_dict(torch.load(CUSTOM_MODEL_PATH))

TypeError: __init__() missing 1 required positional argument: 'device'

# Ensemble and Predict on the Set

In [14]:
def predict(model, images):
    model.eval()
    with torch.no_grad():
        predicted1 = torch.sigmoid(model(images)).cpu()
        predicted2 = torch.sigmoid(model(images.flip(-1))).cpu()
        predicted_logits = (predicted1 + predicted2) / 2
    return predicted_logits

In [15]:
def predict_custom(model, images):
    model.eval()
    with torch.no_grad():
        predicted1, _, _, _, _, _ = model(images)
        predicted1 = torch.sigmoid(predicted1).cpu()
        predicted2, _, _, _, _, _ = model(images.flip(-1))
        predicted2 = torch.sigmoid(predicted2).cpu()
        predicted_logits = (predicted1 + predicted2) / 2
    return predicted_logits

In [16]:
def make_predictions(test_dataloader):
    '''
    performs an ensemble prediction on SEResNet, ResNet, and Custom ResNet
    '''
    predictions = {"StudyInstanceUID": []}
    for title in target_cols:
        predictions[title] = []
    # Iterate over the test_dataloader
    for images, ids in tqdm(test_dataloader):
        images = images.to(device)
        predictedResNet = predict(resnet200D, images) * RESNET200D_WEIGHT
        predictedSE = predict(seresnet152, images) * SERESNET_WEIGHT
        predictedCustom = predict_custom(CustomModel, images) * CUSTOMMODEL_WEIGHT
        # Ensemble Logits
        final_predictions = (predictedResNet + predictedSE + predictedCustom) / DIVIDE # (B, 11)
        # Add to predictions
        predictions['StudyInstanceUID'] += list(ids)
        for title_idx in range(len(target_cols)):
            predicted = final_predictions[:, title_idx]
            title = target_cols[title_idx]
            predictions[title] += predicted.numpy().tolist()
        break
    return predictions

In [17]:
def make_submission(predictions):
    '''
    Creates a pandas daatframe and makes a final prediction
    '''
    df = pd.DataFrame(predictions, index = "StudyInstanceUID")
    df.to_csv("./submission.csv", index = False)

In [18]:
predictions = make_predictions(test_dataloader)
make_submission(predictions)

  0%|          | 0/448 [00:00<?, ?it/s]

NameError: name 'CustomModel' is not defined